In [1]:

def check_gpu(verbose = 1):
    import os

    if verbose:
        print ('\n0. Nvidia Hardware (lspci | grep -i nvidia) : ', [each for each in os.popen('lspci | grep -i nvidia')])
        print ('0. Nvidia Driver (cat /proc/driver/nvidia/version) : ', [each for each in os.popen('cat /proc/driver/nvidia/version')][0])
        print ('0. Nvidia Driver (ls /usr/lib | grep nvidia-) : ', [each for each in os.popen('ls /usr/lib | grep nvidia-')])
        print ('0. Nvidia Driver : (dpkg --get-selections | grep nvidia) : ')
        for each in os.popen('dpkg --get-selections | grep nvidia'):
            print (each)
        print ('\n0. Envs  : CUDA_HOME', os.environ['CUDA_HOME'])
        print ('0. Envs  : CUDA_ROOT',os.environ['CUDA_ROOT'])
        print ('0. Envs  : LD_LIBRARY_PATH:', os.environ['LD_LIBRARY_PATH'])
        print ('0. Envs  : PATH (containing cuda)', [each for each in os.environ['PATH'].split(':') if each.find('cuda') > -1])
        print ('0. CUDnn :', [each.replace('\n', '') for each in os.popen('find /usr/local/cuda-8.0/ -name *dnn*')])

    from keras import backend as K
    K.clear_session()

    print ('\n0. Keras backend:', K.backend())
    if K.backend() == 'tensorflow':
        from tensorflow.python.client import device_lib
        devices = device_lib.list_local_devices()
        for device in devices:
            print ('0. TensorFlow Devices:', str(device.name).replace('\n',''))
        print ('\n')

        return 1 if len(devices) > 1 else 0

    else:
        return 0

# ORIGINAL SOURCE CODE
[Github](https://github.com/mborgerding/onsager_deep_learning)

In [2]:
from __future__ import division
from __future__ import print_function

if __name__ == "__main__":
    if (check_gpu(verbose=0)):
        #!/usr/bin/python
        
        """
        This file serves as an example of how to 
        a) select a problem to be solved 
        b) select a network type
        c) train the network to minimize recovery MSE
        """
        import numpy as np
        import os

        os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # BE QUIET!!!!
        import tensorflow as tf

        np.random.seed(1) # numpy is good about making repeatable output
        tf.set_random_seed(1) # on the other hand, this is basically useless (see issue 9171)

        # import our problems, networks and training modules
        # from tools import problems,networks,train
        import learned_amp_problems as problems
        import learned_amp_train as train
        import learned_amp_networks as networks

        # 1. Create the basic problem structure.
        prob = problems.bernoulli_gaussian_trial(kappa=None,M=250,N=500,L=1000,pnz=.1,SNR=40) #a Bernoulli-Gaussian x, noisily observed through a random matrix
        #prob = problems.random_access_problem(2) # 1 or 2 for compressive random access or massive MIMO
        print ('1. Problem created : bernoulli_gaussian_trial')
        print ('A:', prob.A.shape)
        print ('X:', prob.xval.shape, ' prob.x_:', prob.x_)
        print ('y:', prob.yval.shape, ' prob.y_:', prob.y_)
        print ('Samples for training:', prob.xval.shape[1])

        ## build a LAMP network to solve the problem and get the intermediate results so we can greedily extend and then refine(fine-tune)
        T = 12
        layers = networks.build_LAMP(prob,T=T,shrink='bg',untied=False)
        print ('\n2. Network Built : LAMP with ', T, ' layers')
        for i, layer in enumerate(layers):
            print (i, ' : ', layer)

        # plan the learning
        print ('\n3. Turning layers into training stages')
        training_stages = train.setup_training(layers,prob,trinit=1e-3,refinements=(.5,.1,.01) )

        ## do the learning (takes a while)
        sess = train.do_training(training_stages,prob,'LAMP_bg_giid.npz', maxit = 30000)
    

Using TensorFlow backend.



0. Keras backend: tensorflow
0. TensorFlow Devices: /cpu:0
0. TensorFlow Devices: /gpu:0


1. Problem created : bernoulli_gaussian_trial
A: (250, 500)
X: (500, 1000)  prob.x_: Tensor("x:0", shape=(500, ?), dtype=float32)
y: (250, 1000)  prob.y_: Tensor("y:0", shape=(250, ?), dtype=float32)
Samples for training: 1000

2. Original theta: (1, 2.1972245773362196)
2. Eta function returns: xhat Tensor("truediv_4:0", shape=(500, ?), dtype=float32)  dxdr_ Tensor("Mean:0", shape=(?,), dtype=float32)

2. Network Built : LAMP with  12  layers
0  :  ('Linear', <tf.Tensor 'MatMul_1:0' shape=(500, ?) dtype=float32>, None)
1  :  ('LAMP-bg T=1', <tf.Tensor 'truediv_4:0' shape=(500, ?) dtype=float32>, (<tf.Variable 'theta_0:0' shape=(2,) dtype=float32_ref>,))
2  :  ('LAMP-bg non-linear T=2', <tf.Tensor 'truediv_10:0' shape=(500, ?) dtype=float32>, (<tf.Variable 'theta_1:0' shape=(2,) dtype=float32_ref>,))
3  :  ('LAMP-bg non-linear T=3', <tf.Tensor 'truediv_16:0' shape=(500, ?) dtype=float32>, (<tf.Va

KeyboardInterrupt: 

# EXPERIMENTATION